In [26]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import tensorflow_data_validation as tfdv

warnings.filterwarnings("ignore")

## Analisi esplorativa dei dati

In [35]:
df = pd.read_csv('cs-training-nonull.csv', index_col = 0)

In [37]:
stats = tfdv.generate_statistics_from_dataframe(df)

In [38]:
tfdv.visualize_statistics(stats)

In [47]:
schema = tfdv.infer_schema(stats)

In [48]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'RevolvingUtilizationOfUnsecuredLines',FLOAT,required,,-
'age',INT,required,,-
'NumberOfTime30-59DaysPastDueNotWorse',INT,required,,-
'DebtRatio',FLOAT,required,,-
'MonthlyIncome',FLOAT,required,,-
'NumberOfOpenCreditLinesAndLoans',INT,required,,-
'NumberOfTimes90DaysLate',INT,required,,-
'NumberRealEstateLoansOrLines',INT,required,,-
'NumberOfTime60-89DaysPastDueNotWorse',INT,required,,-


In [45]:
# Assume that other_path points to another TFRecord file
other_stats = tfdv.generate_statistics_from_csv('cs-training.csv')
anomalies = tfdv.validate_statistics(statistics=other_stats, schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'\'\'',New column,New column (column in data but not in schema)
'NumberOfDependents',Expected data of type: FLOAT but got STRING,
'SeriousDlqin2yrs',New column,New column (column in data but not in schema)
'MonthlyIncome',Expected data of type: FLOAT but got STRING,
'isna_mi',Column dropped,Column is completely missing
'isna_nod',Column dropped,Column is completely missing


In [ ]:
print("\033[1m"+'% di record appartenenti alla classe 1:'+"\033[0m",100*len(df[df['SeriousDlqin2yrs'] == 1])/len(df),'%')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
summary = df.describe() # analisi sommaria
summary

###### Cardinalità delle features

In [ ]:
for column in df.columns:   
    print(column)
    print(df[column].nunique())

###### Quali colonne hanno valori nulli?

In [ ]:
print("\033[1m"+"# di valori nulli per feature:\n\n"+"\033[0m")
print(df.isnull().sum()) 

In [ ]:
print("\033[1m"+'% righe con valori null in MonthlyIncome:'+"\033[0m",df['MonthlyIncome'].isnull().sum()/len(df))
print("\033[1m"+'% righe con valori null in NumberOfDependents:'+"\033[0m",df['NumberOfDependents'].isnull().sum()/len(df)) 

### Matrice di Correlazione

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))  
plt.title("Matrice di Correlazione", fontsize =20)
#ax.set_title('Matrice di Correlazione')
sns.heatmap(df.corr(method ='pearson'), linewidths=0.01,annot = True, square=True,linecolor='black',vmin = -1, vmax = 1, ax=ax, cmap = "RdBu")

Non sembra esserci grossa correlazione fra le variabili, eccezion fatta per le 3 colonne relative al numero dei pagamenti in ritardo, che presentano fortissima correlazione.

### Boxplot e Distplot

In [ ]:
def plotchart(col):
    fix, (ax1,ax2) = plt.subplots(1,2,figsize=(20,5))
    #fig.subplots_adjust(hspace=20)
    sns.boxplot(col, orient = 'v', ax=ax1)
    ax1.set_ylabel=col.name
    ax1.set_title('Boxplot di {}'.format(col.name))
    sns.distplot(col,ax=ax2)
    matplotlib.pyplot.subplots_adjust(wspace=.5)
    ax2.set_title('Distribuzione di {}'.format(col.name))

In [ ]:
df.apply(plotchart, axis=0)

In [ ]:
df['MonthlyIncome'].value_counts() # come è distribuita la variabile 'MonthlyIncome'?

Vediamo che 'MonthlyIncome' presenta un numero rilevante di record uguali a 0

### Analisi outliers

In [ ]:
def findoutliers(column,k):
    outliers = []
    Q1 = column.quantile(.25)
    Q3 = column.quantile(.75)
    IQR = Q3-Q1
    lower_lim = Q1-(k*IQR)  
    upper_lim = Q3+(k*IQR)  
    for x in column:
        if x<lower_lim or x>upper_lim:
            outliers.append(x)
    return len(outliers)

In [ ]:
#args={k:1.5}
outliers_count = df.apply(findoutliers, args=(3,), axis=0) ## k=3.0
np.array(outliers_count)

In [ ]:
summary_withOutliers = summary.append(pd.Series(outliers_count, name='outliers_count'))
summary_withOutliers

### Considerazioni
Il dataset è notevolmente sbilanciato (94%-6%), perciò sarà necessario una procedura di bilanciamento durante il training.

Le due colonne con valori null sono molto diverse. Mentre per la colonna "NumberOfDependents" si potrebbe tranquillamente optare per l'eliminazione per righe (poco meno del 3% dei record), lo stesso non può essere fatto per la colonna "MonthlyIncome" (quasi 20% di valori null), bisogna cercare un approccio alternativo; l'approccio alternativo difficilmente potrà essere quello di sostituire i valori null con la media della feature, poichè essa presenta una deviazione standard addirttura più elevata della media. Inoltre, la colonna "MonthlyIncome" presenta un numero elevato di valori uguali a 0.0; ciò va analizzato ulteriormente. 
'MonthlyIncome' non sembra essere nemmeno particolarmente correlata con le altre features, perciò anche una procedura di interpolazione potrebbe essere complicata da applicare.



##### Preparazione del dataset per il training

In [ ]:
prova = df.copy()

Bisogna eliminare tutti gli outlier in 'DebtRatio', perchè presentano grosse incongruenze relativamente a 'MonthlyIncome'.

In [ ]:
query = prova[(prova['DebtRatio'] > 3489.025) & (prova['SeriousDlqin2yrs'] == prova['MonthlyIncome'])]
prova = prova.drop(prova[(prova['DebtRatio'] > 3489.025)].index) 
#prova = prova.drop(query.index)                                                

Rimpiazziamo i valori 96 e 98 nelle 3 colonne (incongruenti con la loro cardinalità) con valori ricalibrati verso il basso, in modo da attenuare l'effetto di questi record

In [ ]:
prova['NumberOfTimes90DaysLate'] = prova['NumberOfTimes90DaysLate'].replace(to_replace=[96,98],value=[20,22])
prova['NumberOfTime60-89DaysPastDueNotWorse'] = prova['NumberOfTime60-89DaysPastDueNotWorse'].replace(to_replace=[96,98],value=[20,22])
prova['NumberOfTime30-59DaysPastDueNotWorse'] = prova['NumberOfTime30-59DaysPastDueNotWorse'].replace(to_replace=[96,98],value=[20,22])
prova

La colonna RUUL rappresenta il tasso di denaro dovuto rispetto al limite di credito; sembra essere affetta da bias, poichè dovrebbe essere in teoria direttamente correlata alla probabilità di andare in default, ma al contrario se analizziamo la distribuzione del target negli outliers, i record che presentano default non sono distribuiti come nella fascia di RUUL che va da 1 a 10, quando in teoria dovrebberp essere ancor maggiori in proporzione.

In [ ]:
prova = prova.drop(prova[(prova['RevolvingUtilizationOfUnsecuredLines'] > 10)].index)                            
prova

In [ ]:
prova.isnull().sum()

Dobbiamo sostituire i valori null nelle colonne che presentano tali valori. 

Per quanto riguarda 'MonthlyIncome' optiamo per la mediana, in quanto la media è influenzata dai record relativi a redditi molto elevati.

Invece, per 'NumberOfDependents', optiamo per il valore di moda (ovvero 0), poichè è ragionevole pensare che i record che presentano valori null rappresentano persone, che perciò non hanno alcun dipendente a carico, o comunque aziende con 0 dipendenti.

In [ ]:
median = df['MonthlyIncome'].median()
mode = df['NumberOfDependents'].mode()

condition = (prova['MonthlyIncome'].isnull())

prova['isna_mi'] = 0
prova.loc[condition, 'isna_mi'] = 1
prova.loc[condition, 'MonthlyIncome'] = median

condition = (prova['NumberOfDependents'].isnull())

prova['isna_nod'] = 0

prova.loc[condition, 'isna_nod'] = 1
prova['NumberOfDependents'] = prova['NumberOfDependents'].fillna(value=int(mode), axis=0)

In [ ]:
prova.to_csv('cs-training-nonull.csv',index = False)